In [1]:
import tempfile
import numpy as np
import random as rand
import pandas as pd
import networkx as nx

# Import Stuffs
from bokeh.plotting import figure, show, save, from_networkx
from bokeh.layouts import column, row
from bokeh.models import HoverTool, TapTool, ColumnDataSource, Circle, MultiLine, Range1d, EdgesAndLinkedNodes, NodesAndLinkedEdges, CustomJS, Div
from bokeh.events import Tap

from bokeh.io import show, output_notebook
from bokeh.resources import INLINE

from IPython.display import IFrame, display, HTML

In [2]:
_imgs = [
    "./_imgs/duck.jpg",
    "./_imgs/frogman.png",
    "./_imgs/panda.jpg",
]

imgs=[
    'https://docs.bokeh.org/static/snake.jpg',
    'https://docs.bokeh.org/static/snake2.png',
    'https://docs.bokeh.org/static/snake3D.png',
    'https://docs.bokeh.org/static/snake4_TheRevenge.png',
    'https://docs.bokeh.org/static/snakebite.jpg'
]

fonts=[
    '<i>italics</i>',
    '<pre>pre</pre>',
    '<b>bold</b>',
    '<small>small</small>',
    '<del>del</del>'
]

In [3]:
def extract_packet(row):
    return {"source": row["Source"], "dest": row["Destination"], "protocol": row["Protocol"]}

In [4]:
def build_graph(packets):
    _g = nx.Graph()
    for p in capture.apply(extract_packet, axis = 1):
        _s = p["source"]
        _d = p["dest"]
        if not _g.has_edge(_s, _d):
            _g.add_edge(_s, _d, connections = [])
        _e = _g[_s][_d]
        _e["connections"].append({"protocol": p["protocol"]})
    
    _attrib = {
        n: {
            "name": n,
            "count": len(_g[n]),
            "imgs": rand.choice(imgs),
            "font": rand.choice(fonts),
        } 
        for n in _g.nodes
    }
    
    nx.set_node_attributes(_g, _attrib)
        
    return _g

In [5]:
capture = pd.read_csv("capture.csv")
capture.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.0,65.246.5.18,192.168.1.162,UDP,1292,443 > 50854 Len=1250
1,2,0.0,65.246.5.18,192.168.1.162,UDP,1292,443 > 50854 Len=1250
2,3,0.0,65.246.5.18,192.168.1.162,UDP,1292,443 > 50854 Len=1250
3,4,0.0,65.246.5.18,192.168.1.162,UDP,1292,443 > 50854 Len=1250
4,5,0.0,65.246.5.18,192.168.1.162,UDP,1292,443 > 50854 Len=1250


In [6]:
# function for the custom display and custom callback
def display_event(div, selected=[], attributes=[], style='float:left;clear:left;'):
    return CustomJS(args=dict(div=div, source=selected), code="""
        // mark graph (circle)
        var data = source.data;
        data['x'].push(cb_obj.x);
        data['y'].push(cb_obj.y);
        console.log(data);
        source.change.emit();
        // display tap event graph coordinates
        var attrs = %s; var args = [];
        for (var i = 0; i<attrs.length; i++) {
            args.push(attrs[i] + '=' + Number(cb_obj[attrs[i]]).toFixed(2));
        }
        var line = "<span style=%r><b>" + cb_obj.event_name + "</b>(" + args.join(", ") + ")</span>\\n";
        var text = div.text.concat(line);
        var lines = text.split("\\n")
        if (lines.length > 35)
            lines.shift();
        div.text = lines.join("\\n");
    """ % (attributes, style))

In [7]:
TOOLTIPS = """
    <div>
        <div>
            <img
                src="@imgs" height="42" alt="@imgs" width="42"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div>
        <div>
            <span style="font-size: 17px; font-weight: bold;">$index</span>
            <span style="font-size: 15px; color: #966;">[@count]</span>
        </div>
        <div>
            <span>@font{safe}</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>
"""

In [8]:
# Create Empty Data Source for Tap Data
selected = ColumnDataSource(
    dict(
        x=[],
        y=[],
    )
)

# Create Bokeh Figure
fig = figure(
    title = "Test Title",
)

G = build_graph(capture.apply(extract_packet, axis = 1))

net_graph = from_networkx(G, nx.spring_layout, scale = 10.0, center = (0, 0))
net_graph.node_renderer.glyph = Circle(size = 15, fill_color = "skyblue")
net_graph.edge_renderer.glyph = MultiLine(line_alpha = 0.5, line_width = 1.0)  
# net_graph.selection_policy = NodesAndLinkedEdges()
net_graph.inspection_policy = EdgesAndLinkedNodes()                              
                                     
fig.renderers = [net_graph]

# Add Tap Tool (for tapping)
fig.add_tools(
    TapTool(),
    HoverTool(tooltips=TOOLTIPS),
)

# Create Div for Tap Feedback (this will sit next too the graph)
div = Div(
    width=400,
    height=fig.plot_height,
    height_policy="fixed",
    style={"padding": "6px", "padding-top": "16px", "font_size":"14px"},
)

# Create Event Callback on Tap and use the CustomJS callback we have defined in the display_event function
fig.js_on_event(Tap, display_event(div, selected=selected, attributes=["x", "y"]))

# Viz Layout
layout = row(fig, div)

In [9]:
output_notebook()
show(layout)

Loading BokehJS ...